<a href="https://colab.research.google.com/github/Parth-349/PRODIGY_GA_01/blob/main/gpt2_finetuning_poetic_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Step 1: Install Required Libraries
!pip install transformers datasets accelerate

In [12]:
# Step 2: Import Libraries
from datasets import load_dataset, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, pipeline

In [13]:
# Step 3: Load Pretrained GPT-2 Tokenizer and Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [14]:
# Step 4: Define Custom Dataset (using the provided text)
text_data = """The knight crossed the valley with fire behind him.
The moonlight lit up her journal as she wrote her last words.
Beneath the waves, ancient ruins whispered tales of betrayal.
Every shadow told a story; every whisper carried a warning.
Let go, for even the river forgets its beginning.
Sometimes the questions matter more than the answers.
You are not broken; you are just unfolding.
Grief is just love without a place to land.
We are all stories pretending to be solid.
Truth waits in the quiet corners where ego dares not look.
"""

text_lines = text_data.strip().split('\n')

In [15]:
# Step 5: Load and Tokenize Dataset
# Create a dataset from the loaded text
dataset = Dataset.from_dict({"text": text_lines})


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [16]:
# Step 6: Prepare Training Components
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    prediction_loss_only=True,
    fp16=True,
    report_to="none",
)

In [17]:
# Step 7: Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-17-1427066281.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=25, training_loss=3.6274209594726563, metrics={'train_runtime': 22.4787, 'train_samples_per_second': 2.224, 'train_steps_per_second': 1.112, 'total_flos': 13064601600000.0, 'train_loss': 3.6274209594726563, 'epoch': 5.0})

In [18]:
# Step 8: Save Model
model.save_pretrained("gpt2-finetuned")
tokenizer.save_pretrained("gpt2-finetuned")

('gpt2-finetuned/tokenizer_config.json',
 'gpt2-finetuned/special_tokens_map.json',
 'gpt2-finetuned/vocab.json',
 'gpt2-finetuned/merges.txt',
 'gpt2-finetuned/added_tokens.json')

In [19]:
# Step 9: Generate Text from the Fine-Tuned Model
generator = pipeline("text-generation", model="gpt2-finetuned", tokenizer=tokenizer)
prompt = "Once upon a time"
output = generator(
    prompt,
    max_new_tokens=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.2
)
print(output[0]['generated_text'])

Device set to use cuda:0


Once upon a time the wind blew through all corners of space like its own sails. For there was still hope after much suffering, but it seemed that even death itself could not keep quiet for long. It would always have to hear his voice as if in warning order: never again did she sing songs with an easy vow – ever before her heart gave up singing from within and began once more listening only above them…
That's when I heard my father say "I'll tell you what will befall us." That
